In [1]:
import numpy as np
import tensorflow as tf

In [2]:
sw = tf.train.SummaryWriter('.', graph=tf.get_default_graph())

Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.


In [3]:
W = tf.get_variable(shape=[1,2], name='weights')

In [4]:
b = tf.get_variable(shape=[1,], name='bias')

In [5]:
x = tf.placeholder(shape=[2,1], dtype=tf.float32, name='x')

In [6]:
y = tf.matmul(W, x) + b


In [7]:
sw.add_graph(graph=tf.get_default_graph())

In [8]:
init = tf.initialize_all_variables()
sess = tf.Session()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [9]:
sess.run(init)

In [10]:
x_in = [[3], [4]]

In [11]:
sess.run(y, feed_dict={x: x_in})

array([[ 1.33641362]], dtype=float32)

In [12]:
y_label = tf.placeholder(shape=[], dtype=tf.float32, name='y_label')
diff = y - y_label
L = tf.reduce_sum(diff * diff)

In [13]:
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(L)

In [14]:
eval_data = np.loadtxt(open("eval_data.csv","rb"), delimiter=",")
def eval():
    acc = 0.
    for x1, x2, y_in in eval_data:
        acc += sess.run(L, feed_dict={x: [[x1],[x2]], y_label: y_in})
    print acc/len(eval_data)
eval()

SyntaxError: invalid syntax (<ipython-input-14-79dc6d7f5feb>, line 6)

In [ ]:
data = np.loadtxt(open("training_data.csv","rb"), delimiter=",")
for x1, x2, y_in in data:
  sess.run(train, feed_dict={x: [[x1],[x2]], y_label: y_in})

In [ ]:
eval()

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)
R = tf.contrib.learn.LinearRegressor(feature_columns=[tf.contrib.layers.real_valued_column('', dimension=2)], 
                                     optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01))
R.fit(x=data[:,0:2], y=data[:,2:3], batch_size=10, max_steps=1000)
R.evaluate(x=eval_data[:,0:2], y=eval_data[:,2:3])
R.predict(x=np.asarray([1.5, 3.4]))

In [ ]:
D = tf.contrib.learn.DNNRegressor(feature_columns=[tf.contrib.layers.real_valued_column('', dimension=2)],
                                  hidden_units=[10,]*9)
D.fit(x=data[:,0:2], y=data[:,2:3], batch_size=10, max_steps=1000)
D.evaluate(x=eval_data[:,0:2], y=eval_data[:,2:3])
D.predict(x=np.asarray([1.5, 3.4]))

In [ ]:
filename_queue = tf.train.string_input_producer(["training_data.csv"], num_epochs=1)
reader = tf.TextLineReader()
key, line = reader.read(filename_queue)
x1,x2,y_in = tf.decode_csv(line, record_defaults=[[0.0], [0.0], [0.0]])
sess.run(tf.group(tf.initialize_all_variables(),
                  tf.initialize_local_variables()))
tf.train.start_queue_runners(sess)

In [ ]:
xq = tf.expand_dims(tf.pack([x1,x2]), 1)
yq = tf.matmul(W, xq) + b
diff = yq - y_in
Lq = tf.reduce_sum(diff * diff)
trainq = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(Lq)

In [ ]:
try:
    for i in xrange(100000):
        sess.run(trainq)
        if i % 1000 == 0:
            print "i %d" % i
            eval()
except tf.errors.OutOfRangeError:
    print "done, i=%d" % i

In [ ]:
eval()

In [ ]:
saver = tf.train.Saver()
saver.save(sess, 'my_checkpoint.ckpt')

In [ ]:
sess.run(init)
eval()

In [ ]:
saver.restore(sess, 'my_checkpoint.ckpt')
eval()